In [1]:
import nltk
import re
from gensim import corpora
from gensim.test.utils import get_tmpfile
from gensim.models import TfidfModel
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import operator
import json
import pandas as pd
from gensim.models import LsiModel
from gensim import similarities
import gensim
import numpy as np
from gensim.corpora import MmCorpus

In [2]:
'''book = open_workbook('ignore/UNSPSC English v220601 project.xlsx')'''
book = open_workbook('ignore/Unspec List2b.xlsx')
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

listOfEntry = []
FamilyList = []
sortlist=[]
sort = []
sen = []
counter = 1
num=1

path = "UnSpec Family"

if not os.path.exists(path):
    os.makedirs(path)



for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    
    if(entry.get("Family")!="" and entry.get("Class")==""):
        #if(counter%2==0):
        if(listOfEntry!=[]):
            f = open(os.path.join(path,"family{0}.txt".format(num)),"w")
            sortlist.append(sort)
            for line in listOfEntry:
              hold = str(line,).strip('[]')
              hold = hold.replace("'","")
              #hold = str(str(line).strip('[]'),encoding='utf-8')
              f.write(json.dumps(hold))
              f.write("\n")
            #f.writelines(json.dumps(listOfEntry))
            counter += 1
            f.close()
            listOfEntry.clear()
            sort=[]
            continue

        #else:
         #   counter += 1
        
    else:
        e = entry.get("Class Title")
        f = entry.get("Class Definition")
        g = entry.get("Commodity Title")
        h = entry.get("Commodity Definition")
        i = entry.get("Family")
        j = entry.get("Commodity")
        if(f != "" and h != ""):
            result = e+". "+f+". "+g+". "+h+". "
        elif(f != "" and h == ""):
            result = e+". "+f+". "+g+". "+h
        elif(f == "" and h != ""):
            result = e+". "+f+" "+g+". "+h+". "
        else:
            result = e + ". " + f + "" + g + ". " + h
        sen = sent_tokenize(result.lower())
        listOfEntry.append(sen)
        sort.append(j)
        num=int(i)
        e, f, g, h = "", "", "", ""

        
f = open(os.path.join(path,"family{0}.txt".format(num)),"w+")
sortlist.append(sort)
for line in listOfEntry:
    hold = str(line,).strip('[]')
    hold = hold.replace("'","")
    f.write(json.dumps(hold))
    f.write("\n")
counter += 1
f.close()
        

stop_words = set(stopwords.words('english'))
file_docs = []


anotherCount = 1
df1 = pd.DataFrame()
df2 = []
fam = []
path1 = os.path.join(path)
for file in os.listdir(path1):
    with open(os.path.join(path,file),'r', encoding='utf-8') as infile:
        txt = infile.readlines()
        for i in txt:
            fam.append(i) 
        df1 = df1.append(fam)
        df2.append(df1)
        df1 = pd.DataFrame()
        fam.clear()
        anotherCount += 1
       
alist =[]  

for data in df2:
    alist.append(data[0].tolist())
    
print(len(alist))

token =[]


Corlist=[]
Dictlist=[]
lengths=[]
inc = 1
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]
for input in alist:
    for j in input:
        tokens = word_tokenize(j)
        tokens = [ j for j in tokens if not j in ch ]
        tokens = [w for w in tokens if not w in stop_words]
        tokens = [p.replace('.',"") for p in tokens]
        token.append(tokens)
    out_fname = get_tmpfile("corpus{0}".format(inc))
    tmp_fname = get_tmpfile("dictionary{0}".format(inc))
    lengths.append(len(token))
    inc += 1
    dictionary = corpora.Dictionary(token)
    corpus = [dictionary.doc2bow(entry) for entry in token]
    token.clear()
    tf_idf = TfidfModel(corpus)
    cor_tf = tf_idf[corpus]
    MmCorpus.serialize(out_fname,cor_tf)
    Corlist.append(out_fname)
    dictionary.save_as_text(tmp_fname)
    Dictlist.append(tmp_fname)
    
    
    
print(lengths)
print(Dictlist)
print(Corlist)

print(sortlist)

3
[43, 40, 159]
['C:\\Users\\z12dr\\AppData\\Local\\Temp\\dictionary1', 'C:\\Users\\z12dr\\AppData\\Local\\Temp\\dictionary2', 'C:\\Users\\z12dr\\AppData\\Local\\Temp\\dictionary3']
['C:\\Users\\z12dr\\AppData\\Local\\Temp\\corpus1', 'C:\\Users\\z12dr\\AppData\\Local\\Temp\\corpus2', 'C:\\Users\\z12dr\\AppData\\Local\\Temp\\corpus3']
[['', 30121501.0, 30121503.0, 30121504.0, '', 30121601.0, 30121602.0, 30121603.0, 30121604.0, 30121605.0, '', 30121701.0, 30121702.0, 30121703.0, 30121704.0, 30121705.0, 30121706.0, 30121707.0, 30121708.0, 30121709.0, 30121710.0, 30121711.0, 30121712.0, 30121713.0, 30121714.0, 30121715.0, 30121716.0, 30121717.0, 30121718.0, 30121719.0, 30121720.0, 30121721.0, '', 30121801.0, 30121802.0, 30121803.0, '', 30121901.0, '', 30122001.0, 30122002.0, 30122003.0, 30122004.0], ['', 39101601.0, 39101603.0, 39101605.0, 39101608.0, 39101609.0, 39101612.0, 39101613.0, 39101614.0, 39101615.0, 39101616.0, 39101617.0, 39101618.0, 39101619.0, 39101620.0, 39101621.0, 39101622

In [3]:
test = "MASS TRANSPORTATION - RAIL VEHICLE PARTS AND ACCESSORIES"
test = test.lower()
query_doc = word_tokenize(test)


listNew = []
listNew2 =[]

def reduceList(lengths,sortlist, dictionarys,corpuses):
    for i in range(len(lengths)):
        mm = MmCorpus(corpuses[i])
        load_dic = corpora.Dictionary.load_from_text(dictionarys[i])
        lsi = LsiModel(mm,num_topics=lengths[i],id2word = load_dic)
        index = gensim.similarities.MatrixSimilarity(lsi[mm],num_features=lengths[i])
        query_doc_bow = load_dic.doc2bow(query_doc, True)
        query_doc_tf_idf = tf_idf[query_doc_bow]
        sim2 = index[lsi[query_doc_tf_idf]]
        sim = list(zip(sortlist[i],index[lsi[query_doc_tf_idf]]))
        sim.sort(key = operator.itemgetter(1),reverse = True)
        sim2 = sorted(enumerate(sim2), key=lambda item: -item[1])
        listNew2.append(sim2[0:5])
        listNew.append(sim[0:5])
        print(sim[0:5])
        print(sim2[0:5])


reduceList(lengths, sortlist,Dictlist,Corlist)
    
       

print(len(listNew))



[(30121703.0, 0.9248624), (30121714.0, 1.4901161e-08), (30121709.0, 1.1175871e-08), (30121705.0, 7.450581e-09), (30121715.0, 7.450581e-09)]
[(13, 0.9248624), (24, 1.4901161e-08), (19, 1.1175871e-08), (15, 7.450581e-09), (25, 7.450581e-09)]
[('', 0.0), (39101601.0, 0.0), (39101603.0, 0.0), (39101605.0, 0.0), (39101608.0, 0.0)]
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
[(39111616.0, 0.5442184), (39111541.0, 0.48754525), (39112013.0, 0.43974388), (39111821.0, 0.40579844), (39111909.0, 0.37342858)]
[(50, 0.5442184), (31, 0.48754525), (119, 0.43974388), (85, 0.40579844), (103, 0.37342858)]
3


C:\Users\z12dr\Anaconda3\lib\site-packages\gensim\matutils.py:779: RuntimeWarning: overflow encountered in double_scalars
  length = 1.0 * math.sqrt(sum(val ** 2 for _, val in vec))


In [4]:
[(13, 0.4744661), (4, 0.37624702), (41, 0.16147709), (18, 0.14059983), (40, 0.099389255)]
[(17, 0.45512533), (24, 0.42233905), (25, 0.2707626), (26, 0.23992425), (16, 0.20316647)]
[(50, 0.5442184), (31, 0.48754525), (119, 0.43974388), (85, 0.40579844), (103, 0.37342858)]

[(50, 0.5442184),
 (31, 0.48754525),
 (119, 0.43974388),
 (85, 0.40579844),
 (103, 0.37342858)]

In [5]:
print(listNew)

[[(30121703.0, 0.9248624), (30121714.0, 1.4901161e-08), (30121709.0, 1.1175871e-08), (30121705.0, 7.450581e-09), (30121715.0, 7.450581e-09)], [('', 0.0), (39101601.0, 0.0), (39101603.0, 0.0), (39101605.0, 0.0), (39101608.0, 0.0)], [(39111616.0, 0.5442184), (39111541.0, 0.48754525), (39112013.0, 0.43974388), (39111821.0, 0.40579844), (39111909.0, 0.37342858)]]
